In [1]:
import pandas as pd
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

df = pd.read_csv('../Dados/Processed/Índice de Inclusão Financeira.csv')

## Árvore de Decisão

dfID = df.iloc[:, 0:2]
dfTarget = df.iloc[:, 2:3]
dfValues = df.iloc[:, 3:]

clf_tree = DecisionTreeRegressor( 
            criterion = "mse", random_state = 500, splitter = 'random', min_samples_split = 4,
            max_depth = 3, min_samples_leaf = 2)

clf_tree.fit(dfValues, dfTarget)

fig, ax = plt.subplots(figsize=(20, 20))  # whatever size you want
plot_tree(clf_tree, ax = ax)#, feature_names = dfValues.columns)#, class_names = dfTarget.columns)
plt.show()

score = clf_tree.score(dfValues,dfTarget)
score

x = []

for i in [30, 10, 16, 22, 19, 36, 20]:
    x = x + [dfValues.iloc[:, i:i+1].columns[0]]
        
dfPlot = dfValues.copy()

dfPlot['Classe'] = 0

a30 = 'Number of electronic service outposts of finance companies_Geo'
a10 = 'Number of credit unions correspondents_Demo'
a16 = 'Number Fintechs Brazil'
a22 = 'Number of ATMs of financial institutions_Geo'
a19 = 'Number of banking service outposts of financial institutions_Geo'
a36 = 'Number Fintechs Brazil - Celular'
a20 = 'Number of electronic service outposts of financial institutions_Geo'

dfPlot.loc[(dfPlot[a30] <= 0.652) & (dfPlot[a10] <= 0.077) & (dfPlot[a22] <= 0.01), 'Classe'] = 0
dfPlot.loc[(dfPlot[a30] <= 0.652) & (dfPlot[a10] <= 0.077) & (dfPlot[a22] >  0.01), 'Classe'] = 1

dfPlot.loc[(dfPlot[a30] <= 0.652) & (dfPlot[a10] >  0.077) & (dfPlot[a19] <= 0.163), 'Classe'] = 2
dfPlot.loc[(dfPlot[a30] <= 0.652) & (dfPlot[a10] >  0.077) & (dfPlot[a19] >  0.163), 'Classe'] = 3

dfPlot.loc[(dfPlot[a30] >  0.652) & (dfPlot[a16] <= 0.79) & (dfPlot[a36] <= 0.51), 'Classe'] = 4
dfPlot.loc[(dfPlot[a30] >  0.652) & (dfPlot[a16] <= 0.79) & (dfPlot[a36] >  0.51), 'Classe'] = 5

dfPlot.loc[(dfPlot[a30] >  0.652) & (dfPlot[a16] >  0.79) & (dfPlot[a20] <= 0.689), 'Classe'] = 6
dfPlot.loc[(dfPlot[a30] >  0.652) & (dfPlot[a16] >  0.79) & (dfPlot[a20] >  0.689), 'Classe'] = 7

pd.concat([dfID, dfPlot[x], dfTarget], axis = 1).to_csv('../Dados/Analysis/ToAnalysis2.csv', index = None)

pd.concat([dfID, dfPlot, dfTarget], axis = 1).to_csv('../Dados/Analysis/ToAnalysisALL.csv', index = None)

dfPlot.to_csv('../Dados/Analysis/ToAnalysis.csv', index = None)

dfPlot[dfPlot.Classe.isin([4,5,6,7])].corr()

# Classificador

def classifier(value):
    if (value <= 1) & (value > 0.75):
        return 'High'
    elif (value < 0.75) & (value >= 0.5):
        return 'Above Average'
    elif (value < 0.5) & (value >= 0.25):
        return 'Moderate'
    else:
        return 'Low'

dfTarget['Índice de Inclusão Financeira'].value_counts()

dfID = df.iloc[:, 0:2]
dfTarget = df.iloc[:, 2:3]
dfTarget['Índice de Inclusão Financeira'] = dfTarget['Índice de Inclusão Financeira'].apply(lambda value : classifier(value))
dfValues = df.iloc[:, 3:]

clf_tree = DecisionTreeClassifier( 
            criterion = "entropy", random_state = 100, 
            max_depth = 3, min_samples_leaf = 2)

clf_tree.fit(dfValues, dfTarget)

fig, ax = plt.subplots(figsize=(20, 20))  # whatever size you want
plot_tree(clf_tree, ax = ax)
plt.show()

